### QR Decomposition    

In [1]:
import pyJvsip as pjv
f='%.5f'

Solve using QR Class

Create some data $\vec{{\vec{A}}}*\vec{x} = \vec{b}$   
Note we create an $\vec{x}$ and calculate a $\vec{b}$ directly

In [2]:
m=7;n=5
A=pjv.create('mview_d',m,n).randn(5)
x=pjv.create('mview_d',n,1).randn(9)
print('Matrix A');A.mprint(f)
print('Known x vector');x.transview.mprint(f)
b=A.prod(x)
print('Calculated b=Ax vector');b.transview.mprint(f)

Matrix A
[ 0.50802  0.53515  0.69864 -0.96027  0.23142;
  0.04042 -0.47661  0.20765  0.50621 -0.38285;
  0.15746  0.78115 -0.96815 -0.32034  0.79250;
  0.79172 -0.25782  0.12663  1.35454  0.25523;
 -0.19460  0.34111 -0.49602  0.17191  1.62412;
  0.19054  0.63559 -0.47360  0.83529  0.02671;
 -0.12147 -0.71231  0.31025 -1.31418  0.09632]

Known x vector
[ 0.39248 -1.35556 -0.24268  1.22453 -0.65029]

Calculated b=Ax vector
[-2.02196  1.48038 -1.66977  2.12221 -1.26404  0.33360 -0.82927]



Note QR overwrites the input matrix; so to preserve our original matrix we use a copy. The QR object will keep a reference to the copy (which means python will not garbage collect it). The QR class has a dictionary `tSel` item which allows the user to query for the proper type based on the associated matrix.

In [3]:
print('Example of QR.tSel: %s'%pjv.QR.tSel[A.type])
qrObj = pjv.QR(pjv.QR.tSel[A.type],m,n,pjv.VSIP_QRD_SAVEQ)
_=qrObj.qrd(A.copy)
print('Solve for x estimate using b. Done in place. Here we make a copy of b first. A subview is printed since b is longer than the x estimate')
xb = b.copy
qrObj.solve('LLS',xb).transview[:,:A.rowlength].mprint(f)

Example of QR.tSel: qr_d
Solve for x estimate using b. Done in place. Here we make a copy of b first. A subview is printed since b is longer than the x estimate


TypeError: unorderable types: NoneType() > int()

Below we use the create function to produce a qr object with (default) save full Q. We then extract Q using prodQ and the identity matrix. We then check to see if Q is orthonormal.

In [ ]:
B=pjv.create('cmview_d',8,3).randn(10)
qr=pjv.create('cqr_d',8,3)
qr.qrd(B)

In [ ]:
qr.sizeQ

In [ ]:
Q=qr.prodQ('NTRANS','RSIDE',pjv.create('cmview_d',8,8).identity)
Q.mprint('%.2f')
Q.prodh(Q).mprint('%.1f')

Below we do the above example except we use a convenience view method to create the QR object directly from the matrix view; and we use prodQ twice to do the matrix multiply first times the identity to get Q and then Q times it's hermitian.

In [ ]:
qr=pjv.create('cmview_d',8,3).randn(10).qr
Q=qr.prodQ('NTRANS','RSIDE',pjv.create('cmview_d',8,8).identity)
qr.prodQ('HERM','RSIDE',Q).mprint('%.2f')

We also have a convenience view method to get the Q and R matrices explicitly.

In [ ]:
B=pjv.create('cmview_d',8,3).randn(10)
print('input B');B.mprint('%.2f')
Q,R=B.copy.qrd
print('Q');Q.mprint('%.2f')
print('R');R.mprint('%.2f'); 
print('Frobenius norm of difference of B-Q * R:  %.5f '%(Q.prod(R)-B).normFro)